In [ ]:
!pip install instructlab
!SETUPTOOLS_SCM_PRETEND_VERSION=0.1 pip install --ignore-installed --upgrade ./sdg
# !pip install --upgrade ./sdg
!pip install docling-parse==1.13.0
!pip install docling~=1.16.1


In [1]:
import os
import random
from datasets import load_dataset
from utils.data import postprocess_and_save, pretty_print_dict
from instructlab.sdg.utils.docprocessor import DocProcessor

/Users/cchase/git/github/cfchase/samsung-poc/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setup Instructions

This demo demonstrates the process of converting raw PDF files into InstructLab Synthetic Knowledge Infusion Data using the RBC POC as an example. Follow these steps to get started with your own data.

#### Steps to Get Started:

1. **Organize Your Documents:**
   - Create a new directory under the `document_collection` directory for your specific project. For example, if your project is named "Samsung," your directory structure should look like this:
     ```
     |-- document_collection
     |   `-- Samsung
     |       |-- Samsung_data.pdf
     |       `-- qna.yaml
     ```
   - Place all your PDF files and ICL files (like `qna.yaml`) into this directory.

2. **Format Your ICLs:**
   - Ensure your ICL files contain sufficient context and question-answer pairs. We recommend including at least 5 distinct contexts, each with a minimum of 3 sets of questions and answers. More entries will improve the robustness of your data.
    - The ICL file should be in the following format (refer to the `document_collection/Samsung/qna.yaml` file for an example):

    ```yaml
    domain: 
    document_outline: A one to two line description of the document
    seed_examples:
      - context: <context 1 goes here>
        question_and_answers:
          - question: <question 1 goes here>
            answer: <answer 1 goes here>
          - question: <question 2 goes here>
            answer: <answer 2 goes here>
          - question: <question 3 goes here>
            answer: <answer 3 goes here>
    ... 


   - **Note:** Replace placeholders with actual content relevant to your documents. Ensure the contexts are clear and questions are well-formulated to extract meaningful answers.

3. **Update the Data Directory Path:**
   - In the script or code where the data directory is specified, update the `data_dir` variable to reflect the path to your new directory. For example:
     ```python
     data_dir = "document_collection/Samsung"
     ```
4. **Update the Output Directory Path:**
   - In the script or code where the data directory is specified, update the `output_dir` variable to reflect the path to your directory. For example:
     ```python
     data_dir = "output/Samsung"
     ```
---

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

# Access the variables
data_dir = os.getenv('DATA_DIR')
output_dir = os.getenv('OUTPUT_DIR')
os.makedirs(output_dir, exist_ok=True)

### PDF Documents to Seed Dataset

To convert PDF documents into a usable seed dataset, we employ [Docling](https://github.com/DS4SD/docling), a tool designed for extracting and processing text from PDF files. The text extraction process involves parsing the PDF documents and saving the extracted text into a structured JSON file. The extracted text in JSON format can be used to generate InstructLab Synthetic Knowledge Infusion Data.


#### Step 1: 

Run the following command to extract text from the PDF documents and save it in JSON format:

⚠️ **Note:** This process takes about 5 minutes to run for this example


In [3]:
!echo $data_dir
!python ./sdg/scripts/docparser.py --input-dir {data_dir} --output-dir {output_dir}

document_collection/Samsung2
Fetching 9 files: 100%|████████████████████████| 9/9 [00:00<00:00, 84072.91it/s]


#### Step 2: 

Now that we have extracted the text from the PDF documents, we can proceed to process the extracted data, we do the following:

- Split the extracted text into chunks 
- Populate user provided ICLs with the chunks 

In [4]:
dp = DocProcessor(output_dir, user_config_path=f'{data_dir}/qna.yaml')

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [5]:
seed_data = dp.get_processed_dataset()
seed_data

[10:05:31] INFO     Processing parsed docling json file: output/Samsung2/Samsung SDS            ]8;id=870390;file:///Users/cchase/git/github/cfchase/samsung-poc/venv/lib/python3.11/site-packages/instructlab/sdg/utils/docprocessor.py\docprocessor.py]8;;\:]8;id=556726;file:///Users/cchase/git/github/cfchase/samsung-poc/venv/lib/python3.11/site-packages/instructlab/sdg/utils/docprocessor.py#263\263]8;;\
                    2024Q2_Script_Eng.json                                                                         

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
Filter: 100%|██████████| 70/70 [00:00<00:00, 4118.58 examples/s]


Dataset({
    features: ['document_outline', 'document_title', 'domain', 'icl_document', 'icl_query_1', 'icl_response_1', 'icl_query_2', 'icl_response_2', 'icl_query_3', 'icl_response_3', 'document'],
    num_rows: 70
})

In [6]:
seed_data.to_json(f'{output_dir}/seed_data.jsonl', orient='records', lines=True)

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 478.04ba/s]


344543

In [7]:
pretty_print_dict(f'{output_dir}/seed_data.jsonl')

Generating train split: 70 examples [00:00, 30688.96 examples/s]


{
    "document_outline": "This is a transcript of Samsung SDS's Q2 2024 earnings conference call.\n",
    "document_title": "Samsung SDS 2024Q2_Script_Eng",
    "domain": "finance",
    "icl_document": "Now, let us review the Q2 earnings. Please refer to the first page of\nthe earnings 
presentation slides.\n\nIn Q2, our revenue was KRW 3.37 trillion, a 3.7% increase from the\nprevious quarter and a 
2.4% increase year-over-year.\n\nLooking at the profit, the gross profit was KRW 500 billion, a 1.9%\nincrease from
the previous quarter and a 12.7% improvement\nyear-over-year. The gross profit margin was 14.8%, a 0.3 
percentage\npoint decrease from the previous quarter but a 1.3 percentage point\nimprovement 
year-over-year.\n\nNext is the operating profit for Q2. Operating profit was KRW 221\nbillion, a 2% decrease from 
the previous quarter but a 7% increase\nyear-over-year. The operating profit margin was 6.6%, a 0.4 
percentage\npoint decrease from the previous quarter but a 0.3 percentage point\nincrease year-over-year.\n\nNet 
profit was KRW 190 billion, a 12.2% decrease from the previous\nquarter but an 11.8% increase year-over-year.\n",
    "icl_query_1": "What was Samsung SDS's revenue in Q2 of 2024?\n",
    "icl_response_1": "In the 2nd quarter of 2024, Samsung SDS had a revenue of KRW\n3.37 trillion.\n",
    "icl_query_2": "For Samsung SDS in the second quarter of 2024, what was the gross\nprofit?\n",
    "icl_response_2": "Samsung SDS had a gross profit of KRW 500 billion in Q2 of 2024.\nThis was a 1.9% increase 
from the preceding quarter, and\nrepresents a 12.7 improvement year-over-year.\n",
    "icl_query_3": "Did Samsung SDS see net profit growth in Q2 2024?\n",
    "icl_response_3": "The answer to this is complex. In one sense, net profit grew, but\nin another sense, it did 
not grow. Samsung SDS had a net profit of\nKRW 190 billion in Q2 of 2024. This was a 12.2% decrease from 
the\nprevious quarter, so it was not growth based on the previous quarter.\nHowever, year-over-year this net profit
represents a 11.8% increase\nyear-over-year, so net profit shows growth year-over-year for Q2 2024.\n",
    "document": "Net profit was KRW 190 billion, a 12.2% decrease from the previous quarter but an 11.8% increase 
year-over-year.\n\nNext, let's review the performance by business division on page 2.\n\nFirst, Q2 IT services 
revenue was KRW 1.59 trillion, a 2.1% increase from the previous quarter and a 5% increase year-over-year. The 
cloud business contributed significantly to this growth, with revenue of KRW 556 billion, a 4.8% increase from the 
previous quarter and a 25% increase year-over-year. The logistics business revenue was KRW 1.78 trillion, a 5.3% 
increase from the previous quarter, maintaining the same level as the previous year.\n\nThe operating profit for IT
services in Q2 was KRW 198 billion, an approximate 17% increase from both the previous quarter and the same period 
last year. The operating profit margin was 12.5%, a 1.6 percentage point increase from the previous quarter and a 
1.3 percentage point improvement year-over-year.\n\nThe logistics business recorded an operating profit of KRW 23 
billion with an operating profit margin of 1.3%. The decline in logistics operating profit margin in Q2 is due to 
significant one-time gains in Q1 and reduced profitability from decreased air transport revenue in Q2.\n\nThis 
concludes the brief overview of Q2 performance.\n\nAdditionally, regarding Samsung SDS's ESG, we published and 
distributed the 2023 Samsung SDS Sustainability Report on our website on June 30, explaining our ESG performance. 
Please refer to this newly published report to understand our ESG management performance and strategies.\n\nNow, 
let me open the floor for division leaders to explain the performance of Q2 and outline any upcoming plans.\n\n## 
**[Strategic Marketing Office]**"
}

### Convert JSONL to markdown files

In [8]:
import pandas as pd
import os
import json

# Create the output directory if it doesn't exist
md_output_dir = f"{output_dir}/md"
os.makedirs(md_output_dir, exist_ok=True)


In [9]:
def save_document(index, document_text):
    file_name = f"document_{index+1}.md"
    file_path = os.path.join(md_output_dir, file_name)
    
    with open(file_path, 'w') as f:
        f.write(document_text)
    
    print(f"Saved {file_path}")


In [10]:
jsonl_file_path = f"{output_dir}/seed_data.jsonl"

In [11]:

with open(jsonl_file_path, 'r') as f:
    for i, line in enumerate(f):
        entry = json.loads(line)
        # Use a unique index for each file
        document_text = entry.get('document', '')
        save_document(i, document_text)

Saved output/Samsung2/md/document_1.md
Saved output/Samsung2/md/document_2.md
Saved output/Samsung2/md/document_3.md
Saved output/Samsung2/md/document_4.md
Saved output/Samsung2/md/document_5.md
Saved output/Samsung2/md/document_6.md
Saved output/Samsung2/md/document_7.md
Saved output/Samsung2/md/document_8.md
Saved output/Samsung2/md/document_9.md
Saved output/Samsung2/md/document_10.md
Saved output/Samsung2/md/document_11.md
Saved output/Samsung2/md/document_12.md
Saved output/Samsung2/md/document_13.md
Saved output/Samsung2/md/document_14.md
Saved output/Samsung2/md/document_15.md
Saved output/Samsung2/md/document_16.md
Saved output/Samsung2/md/document_17.md
Saved output/Samsung2/md/document_18.md
Saved output/Samsung2/md/document_19.md
Saved output/Samsung2/md/document_20.md
Saved output/Samsung2/md/document_21.md
Saved output/Samsung2/md/document_22.md
Saved output/Samsung2/md/document_23.md
Saved output/Samsung2/md/document_24.md
Saved output/Samsung2/md/document_25.md
Saved out